Checking the GPU Type

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Aug  9 23:52:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.41       Driver Version: 471.41       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:08:00.0  On |                  N/A |
| 35%   58C    P2   140W / 300W |  10967MiB / 11264MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Checking RAM Size

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 68.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
! pip install -q datasets transformers

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\tqdm-4.61.1.dist-info\\direct_url.json'
Consider using the `--user` option or check the permissions.



In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [ ]:
import transformers

print(transformers.__version__)

4.21.1


Fine-tuning begins

In [ ]:
model_checkpoint = "nlptown/bert-base-multilingual-uncased-sentiment"
#model_checkpoint = "distilbert-base-uncased"
#model_checkpoint = "bert-base-cased"
batch_size = 16

Loading the datasets

In [ ]:
from datasets import load_dataset, load_metric, list_metrics

In [ ]:
actual_task = "mnli"
metric = load_metric('glue', actual_task)

Mounting the Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

% cd /content/drive/MyDrive/datasets

Mounted at /content/drive
/content/drive/MyDrive/datasets


Reading in the Amazon Product Review datasets

In [ ]:
import datasets
import random
import pandas as pd
import numpy as np

path = ['amazon_reviews_us_Apparel_v1_00.tsv',
        'amazon_reviews_us_Automotive_v1_00.tsv',
        'amazon_reviews_us_Baby_v1_00.tsv',
        'amazon_reviews_us_Beauty_v1_00.tsv',
        'amazon_reviews_us_Books_v1_02.tsv',
        'amazon_reviews_us_Camera_v1_00.tsv',
        'amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv',
        'amazon_reviews_us_Digital_Software_v1_00.tsv',
        'amazon_reviews_us_Digital_Video_Games_v1_00.tsv',
        'amazon_reviews_us_Electronics_v1_00.tsv',
        'amazon_reviews_us_Furniture_v1_00.tsv',
        'amazon_reviews_us_Grocery_v1_00.tsv',
        'amazon_reviews_us_Health_Personal_Care_v1_00.tsv',
        'amazon_reviews_us_Mobile_Electronics_v1_00.tsv',
        'amazon_reviews_us_Outdoors_v1_00.tsv',
        'amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv',
        'amazon_reviews_us_Sports_v1_00.tsv',
        'amazon_reviews_us_Tools_v1_00.tsv']

li = []
num_each = 2000

for filename in path:
    df = pd.read_csv(filename, index_col=None, header=0, sep="\t", error_bad_lines=False, warn_bad_lines=False)
    df = df.sample(n=num_each)
    li.append(df)
    print(f"Dataset loaded successfully : {filename} ")

product = pd.concat(li, axis=0, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


Dataset loaded successfully : amazon_reviews_us_Apparel_v1_00.tsv 


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Dataset loaded successfully : amazon_reviews_us_Automotive_v1_00.tsv 
Dataset loaded successfully : amazon_reviews_us_Baby_v1_00.tsv 
Dataset loaded successfully : amazon_reviews_us_Beauty_v1_00.tsv 
Dataset loaded successfully : amazon_reviews_us_Books_v1_02.tsv 
Dataset loaded successfully : amazon_reviews_us_Camera_v1_00.tsv 
Dataset loaded successfully : amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv 
Dataset loaded successfully : amazon_reviews_us_Digital_Software_v1_00.tsv 
Dataset loaded successfully : amazon_reviews_us_Digital_Video_Games_v1_00.tsv 
Dataset loaded successfully : amazon_reviews_us_Electronics_v1_00.tsv 
Dataset loaded successfully : amazon_reviews_us_Furniture_v1_00.tsv 
Dataset loaded successfully : amazon_reviews_us_Grocery_v1_00.tsv 
Dataset loaded successfully : amazon_reviews_us_Health_Personal_Care_v1_00.tsv 
Dataset loaded successfully : amazon_reviews_us_Mobile_Electronics_v1_00.tsv 
Dataset loaded successfully : amazon_reviews_us_Outdoors_v1_00.tsv 

In [ ]:
product

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,42350254,R2T292SE2T9DOB,B00J4C0ZJ6,175284095,Cubavera Men's Short Sleeve Blend Embroidered ...,Apparel,4.0,0.0,0.0,N,Y,"Nice island style, cool feel",Great island style. Nice cool feel. When pre...,2015-03-07
1,US,7754205,R1X0EO2X4NEV0Q,B0019FG8FQ,524914958,Fgx Boot Blade Ii Cold Steel 92fbb,Apparel,5.0,0.0,5.0,N,Y,Five Stars,Sweet!,2015-04-29
2,US,17189901,R3NMFG84NGQJ1Y,B00JEQ9EXK,729449707,Utopia Wear Women's Premium Cushion No Show So...,Apparel,5.0,0.0,0.0,N,Y,Five Stars,Good quality for the price.,2015-06-24
3,US,29045562,R3VHF730F60QSR,B0095UZ3EE,858537381,Footed Pajamas - Laid-back Gray Adult Hoodie O...,Apparel,5.0,1.0,1.0,N,Y,Superb!,Package arrived very quickly. PJ's are soooo s...,2014-09-26
4,US,20402688,R7AXDCJ14DYQ8,B00BHILNS2,428739677,Lee Women's Natural-Fit Pull-On Barely Bootcut...,Apparel,4.0,0.0,0.0,N,Y,Lee pull on barely boot jean,Fits very well and so comfy; I love it! Size w...,2014-06-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,US,962938,R8TR5TKCOM8JR,B006KTY4U2,929990845,Columbia River Knives (CRKINOX4S) Stainless St...,Tools,4,0.0,0.0,N,Y,Four Stars,the case bent,2014-10-08
35996,US,21001069,R146V0OVHVE9EC,B00012Z0ZM,970080139,Fluke Corporation 322 AC Clamp Meter,Tools,5,1.0,1.0,N,Y,Fluke Clamp-on,The very best price I could find and received ...,2011-10-15
35997,US,44850956,RQVWPVF18WBOD,B003BZ719O,817244350,"Pelican 2360 LED Tactical Light, Black",Tools,5,0.0,0.0,N,Y,Excellent light for $35,I bought this light after about 4 hrs of resea...,2012-02-26
35998,US,31117635,R2U7J2PM8CQMW1,B00005QEVQ,793410684,PORTER-CABLE 690LR 11-Amp Fixed-Base Router,Tools,5,0.0,1.0,N,Y,great price and came in great condition,"Finally a new router, great price and came in ...",2015-07-08


In [ ]:
product['review'] = product['review_headline'] + ". " + product['review_body']
product = product[['review', 'star_rating']]
product = product.rename(columns={'review': 'text', 'star_rating': 'label'})
alled = product

Splitting the data into training. validation, testing and downsampling it for training

In [ ]:
from sklearn.model_selection import train_test_split

frac_train = 0.9
frac_test = 0.8

train_all, test = train_test_split(alled, train_size=frac_train, shuffle=True)
train, dev = train_test_split(train_all, train_size=frac_test, shuffle=True)

train = train.dropna()
dev = dev.dropna()
test = test.dropna()
print('Before training shape:',train.shape)
print('Before dev shape:',dev.shape)
print('Before test shape:',test.shape)

'''
train = train.sample(n=train_count)
dev = dev.sample(n=dev_count)
test = test.sample(n=test_count)

print('After training shape:',train.shape)
print('After dev shape:',dev.shape)
print('After test shape:',test.shape)
'''

train.reset_index(drop=True, inplace=True)
dev.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

train = train.astype({'label': int})
dev = dev.astype({'label': int})
test = test.astype({'label': int})

train['label'] = train['label'] - 1
dev['label'] = dev['label'] - 1
test['label'] = test['label'] - 1

# Save each data
train.to_csv("train.csv", index=False)
dev.to_csv("dev.csv", index=False)
test.to_csv("test.csv", index=False)

Before training shape: (25917, 2)
Before dev shape: (6479, 2)
Before test shape: (3600, 2)


load the downsampled dataset for training, validation and testing 

In [ ]:
dataset = load_dataset("csv", data_files={'train': 'train.csv', 'validation': 'dev.csv', 'test': 'test.csv'})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a35b28ae594e6cdc/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset["train"][20]

{'label': 2, 'text': 'Three Stars. very slow to ship,  will not order again'}

Preprocessing 

In [ ]:
from transformers import AutoTokenizer

print(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

nlptown/bert-base-multilingual-uncased-sentiment


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/26 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

Fine-tuning the model

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

print(model_checkpoint)
num_labels = 5
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

nlptown/bert-base-multilingual-uncased-sentiment


In [ ]:
metric_name = "accuracy"
print(metric_name)
model_name = model_checkpoint.split("/")[-1]
print(model_name)

args = TrainingArguments(
    f"{model_name}-finetuned-{actual_task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

accuracy
bert-base-multilingual-uncased-sentiment


Defining the trainer

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/drive/MyDrive/datasets/bert-base-multilingual-uncased-sentiment-finetuned-mnli is already a clone of https://huggingface.co/ak630/bert-base-multilingual-uncased-sentiment-finetuned-mnli. Make sure you pull the latest changes with `repo.git_pull()`.


Training begins!!!

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25917
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3240


Check whether model was trained properly or not

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6480
  Batch size = 16


{'epoch': 2.0,
 'eval_accuracy': 0.7945987654320987,
 'eval_loss': 0.5396403670310974,
 'eval_runtime': 242.4114,
 'eval_samples_per_second': 26.731,
 'eval_steps_per_second': 1.671}

Evaluate the trained model on our held-out test dataset

In [ ]:
predictions = trainer.predict(encoded_dataset["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3599
  Batch size = 16


(3599, 5) (3599,)


{'accuracy': 0.783550986385107}

Compute F1-score on the test dataset

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_true=predictions.label_ids, y_pred=preds, average='weighted')


NameError: ignored

Viewing the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

array = confusion_matrix(y_true=predictions.label_ids, y_pred=preds)

df_cm = pd.DataFrame(array, index = [i for i in [1,2,3,4,5]],
                  columns = [i for i in [1,2,3,4,5]])

plt.figure(figsize = (10,7))
ax = plt.axes()
g = sn.heatmap(df_cm, annot=True, fmt="d", linewidths=.5, ax=ax, annot_kws={"size": 20});
g.set_yticklabels(g.get_yticklabels(), rotation = 0, fontsize = 18)
g.set_xticklabels(g.get_xticklabels(), rotation = 0, fontsize = 18)
ax.set_title(f'confusion matrix on the held-out test set of {len(preds)} rows', fontsize=18)
plt.show()

In [ ]:
trainer.push_to_hub()

In [ ]:
model.push_to_hub("https://huggingface.co/LiYuan/distilbert-base-uncased-finetuned-mnli")